**Add Functions**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score
from sklearn import tree

In [ ]:
!pip install vecstack

  Preparing metadata (setup.py) ... done
  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19861 sha256=ecd54841f33d85024902a0ee540220959e59408dcd19e31341637a13aa386fd9
  Stored in directory: /root/.cache/pip/wheels/b8/d8/51/3cf39adf22c522b0a91dc2208db4e9de4d2d9d171683596220
Successfully built vecstack


In [ ]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from collections import Counter #for Smote,

import warnings
warnings.filterwarnings("ignore")

**Add data files**

In [ ]:
#Read training data file
train_data = pd.read_csv(r'/content/sample_data/Cell Phone Churn-TRAIN.csv')
print(train_data.shape)
print(train_data.head())

(1695, 24)
   MOUMO  MOUPMO  MOU3MO  MOUCH1M  MOUCH3M  CUM3MCH  CUSTMOS  LONGD  CALLW  \
0    105      15      99       90      -84        6       11      0      0   
1    999    1200    1145     -201       55     -146        3      1      1   
2      5       2       1        3        1        4       23      0      0   
3    600     456     398      144       58      202       13      0      1   
4      2       0       0        0        0        0        1      0      0   

   LINES  ...  INCOME  SPORTS NATURE ARTS  HRS_TV         TRAVEL        EDUC  \
0      2  ...   51000       B      1    N      10            NaN   Bachelors   
1      2  ...   80000       A      0    N      40       Domestic     Masters   
2      1  ...   80000       C      0    Y      50  International  HighSchool   
3      1  ...   88000       A      1    N       0  International     Masters   
4      1  ...   29000       B      0    N      75       Domestic  HighSchool   

  TOTMOU  TOTCHNG TARGET  
0    219    

In [ ]:
#Data Description
train_data.info()
train_data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 24 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   MOUMO    1695 non-null   int64 
 1   MOUPMO   1695 non-null   int64 
 2   MOU3MO   1695 non-null   int64 
 3   MOUCH1M  1695 non-null   int64 
 4   MOUCH3M  1695 non-null   int64 
 5   CUM3MCH  1695 non-null   int64 
 6   CUSTMOS  1695 non-null   int64 
 7   LONGD    1695 non-null   int64 
 8   CALLW    1695 non-null   int64 
 9   LINES    1695 non-null   int64 
 10  VOICEM   1695 non-null   int64 
 11  CELL     1695 non-null   int64 
 12  CONVB    1695 non-null   object
 13  SEX      1695 non-null   object
 14  INCOME   1695 non-null   int64 
 15  SPORTS   1695 non-null   object
 16  NATURE   1695 non-null   int64 
 17  ARTS     1695 non-null   object
 18  HRS_TV   1695 non-null   int64 
 19  TRAVEL   1406 non-null   object
 20  EDUC     1695 non-null   object
 21  TOTMOU   1695 non-null   int64 
 22  

,MOUMO,MOUPMO,MOU3MO,MOUCH1M,MOUCH3M,CUM3MCH,CUSTMOS,LONGD,CALLW,LINES,VOICEM,CELL,INCOME,NATURE,HRS_TV,TOTMOU,TOTCHNG,TARGET
count,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000
mean,736.866077,864.444248,456.063127,-78.794690,408.333923,328.031858,15.840118,0.573451,0.436578,1.388201,0.566372,0.352802,86267.280236,0.553392,37.958702,2055.600590,328.031858,0.379351
std,1975.038772,2484.040905,1215.196981,1432.517875,2135.107275,2073.909954,13.287256,0.494721,0.496108,0.571130,0.495721,0.477983,67237.818916,0.497288,32.621918,4929.160738,2073.909954,0.485369
min,1.000000,0.000000,0.000000,-9211.000000,-2337.000000,-8177.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,17000.000000,0.000000,0.000000,0.000000,-8177.000000,0.000000
25%,45.000000,15.000000,29.000000,-10.000000,-84.000000,-70.000000,6.000000,0.000000,0.000000,1.000000,0.000000,0.000000,46000.000000,0.000000,5.000000,116.000000,-70.000000,0.000000
50%,105.000000,98.000000,97.000000,19.000000,-3.000000,0.000000,11.000000,1.000000,0.000000,1.000000,1.000000,0.000000,75000.000000,1.000000,35.000000,264.000000,0.000000,0.000000
75%,555.000000,444.000000,598.000000,111.000000,42.000000,54.000000,26.000000,1.000000,1.000000,2.000000,1.000000,1.000000,98000.000000,1.000000,60.000000,1677.000000,54.000000,1.000000
max,14000.000000,16754.000000,12456.000000,8745.000000,16156.000000,13934.000000,50.000000,1.000000,1.000000,3.000000,1.000000,1.000000,320000.000000,1.000000,125.000000,36237.000000,13934.000000,1.000000


In [ ]:
# To check number of null values
train_data.isna().sum()

,0
MOUMO,0
MOUPMO,0
MOU3MO,0
MOUCH1M,0
MOUCH3M,0
CUM3MCH,0
CUSTMOS,0
LONGD,0
CALLW,0
LINES,0


In [ ]:
#Read test data file
test_data = pd.read_csv(r'/content/sample_data/Cell Phone Churn-TEST.csv')

print(test_data.shape)
print(test_data.head())

(375, 24)
   MOUMO  MOUPMO  MOU3MO  MOUCH1M  MOUCH3M  CUM3MCH  CUSTMOS  LONGD  CALLW  \
0    344     232     455      112     -223     -111       11      1      1   
1    555     444     678      111     -234     -123       27      1      1   
2      2       0       4        1       -3       -2        4      0      0   
3   5678    3457    2156     2221     1301     3522       14      1      1   
4    333     122     144      211      -22      189        9      0      1   

   LINES  ...  INCOME  SPORTS NATURE ARTS  HRS_TV         TRAVEL       EDUC  \
0      2  ...   75000       B      1    Y      25  InterGalactic        GED   
1      2  ...  134000      na      1    Y       5       Domestic        PhD   
2      1  ...   53000      na      1    Y     125            NaN  Bachelors   
3      1  ...   33000       A      1    Y      25       Domestic    Masters   
4      1  ...   44000       F      1    N      35  International    Masters   

  TOTMOU  TOTCHNG TARGET  
0   1031     -111  

In [ ]:
#Data Description
test_data.info()
test_data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375 entries, 0 to 374
Data columns (total 24 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   MOUMO    375 non-null    int64 
 1   MOUPMO   375 non-null    int64 
 2   MOU3MO   375 non-null    int64 
 3   MOUCH1M  375 non-null    int64 
 4   MOUCH3M  375 non-null    int64 
 5   CUM3MCH  375 non-null    int64 
 6   CUSTMOS  375 non-null    int64 
 7   LONGD    375 non-null    int64 
 8   CALLW    375 non-null    int64 
 9   LINES    375 non-null    int64 
 10  VOICEM   375 non-null    int64 
 11  CELL     375 non-null    int64 
 12  CONVB    375 non-null    object
 13  SEX      375 non-null    object
 14  INCOME   375 non-null    int64 
 15  SPORTS   375 non-null    object
 16  NATURE   375 non-null    int64 
 17  ARTS     375 non-null    object
 18  HRS_TV   375 non-null    int64 
 19  TRAVEL   304 non-null    object
 20  EDUC     375 non-null    object
 21  TOTMOU   375 non-null    int64 
 22  TO

,MOUMO,MOUPMO,MOU3MO,MOUCH1M,MOUCH3M,CUM3MCH,CUSTMOS,LONGD,CALLW,LINES,VOICEM,CELL,INCOME,NATURE,HRS_TV,TOTMOU,TOTCHNG,TARGET
count,375.000000,375.000000,375.000000,375.000000,375.000000,375.000000,375.000000,375.000000,375.000000,375.000000,375.00000,375.000000,375.000000,375.000000,375.000000,375.000000,375.000000,375.000000
mean,798.832000,863.752000,441.848000,82.338667,421.877333,502.176000,16.962667,0.528000,0.426667,1.405333,0.56000,0.325333,83752.026667,0.618667,39.626667,2102.152000,502.176000,0.373333
std,2201.622312,2400.535328,1033.515234,1553.701797,2145.207276,2458.637116,13.836466,0.499882,0.495254,0.567359,0.49705,0.469125,64518.018480,0.486363,34.999913,4857.459164,2458.637116,0.484336
min,1.000000,0.000000,0.000000,-9211.000000,-2337.000000,-8177.000000,1.000000,0.000000,0.000000,1.000000,0.00000,0.000000,17000.000000,0.000000,0.000000,0.000000,-8177.000000,0.000000
25%,10.000000,15.000000,29.000000,-10.000000,-84.000000,-70.000000,6.000000,0.000000,0.000000,1.000000,0.00000,0.000000,46000.000000,0.000000,5.000000,95.000000,-70.000000,0.000000
50%,105.000000,90.000000,97.000000,19.000000,-3.000000,0.000000,11.000000,1.000000,0.000000,1.000000,1.00000,0.000000,66000.000000,1.000000,35.000000,264.000000,0.000000,0.000000
75%,555.000000,444.000000,598.000000,112.000000,42.000000,54.000000,27.000000,1.000000,1.000000,2.000000,1.00000,1.000000,91000.000000,1.000000,60.000000,1677.000000,54.000000,1.000000
max,14000.000000,16754.000000,12456.000000,8745.000000,16156.000000,13934.000000,50.000000,1.000000,1.000000,3.000000,1.00000,1.000000,320000.000000,1.000000,125.000000,36237.000000,13934.000000,1.000000


In [ ]:
# To check number of null values
test_data.isna().sum()

,0
MOUMO,0
MOUPMO,0
MOU3MO,0
MOUCH1M,0
MOUCH3M,0
CUM3MCH,0
CUSTMOS,0
LONGD,0
CALLW,0
LINES,0


In [ ]:
#Combine Train and test for one Hot Encoding
combined_Data = pd.concat([train_data,test_data], keys=[0,1])

#Do one Hot encoding for categorical features
combined_Data["TERGET"]=combined_Data["TARGET"].map({1:1,0:0})
#combined_Data = pd.get_dummies(combined_Data,columns=categoricalFeatures)
print(combined_Data['TARGET'])

#Separate Train data and test data
X_train = combined_Data.xs(0)
X_test = combined_Data.xs(1)
y_train=X_train["TARGET"]
y_test=X_test["TARGET"]
X_train1=X_train.iloc[:, :-1].copy()
X_test1=X_test.iloc[:, :-1].copy()

0  0      0
   1      1
   2      1
   3      1
   4      0
         ..
1  370    0
   371    1
   372    0
   373    1
   374    1
Name: TARGET, Length: 2070, dtype: int64


In [ ]:
# Combine Train and Test Data
combined_Data = pd.concat([train_data, test_data], keys=[0, 1])

# One-Hot Encoding for all columns
combined_Data = pd.get_dummies(combined_Data)

# Re-map TARGET if needed (ensure target is numeric)
combined_Data["TARGET"] = combined_Data["TARGET"].map({1: 1, 0: 0})

# Check the combined data after one-hot encoding
print(combined_Data.head())  # Just to verify

# Separate Train Data and Test Data
X_train = combined_Data.xs(0)  # Extract train data (index 0)
X_test = combined_Data.xs(1)   # Extract test data (index 1)

# Separate Features (X) and Target (y)
y_train = X_train["TARGET"]  # Target variable for training
y_test = X_test["TARGET"]    # Target variable for testing

# Features (drop the TARGET column)
X_train1 = X_train.drop("TARGET", axis=1).copy()
X_test1 = X_test.drop("TARGET", axis=1).copy()


     MOUMO  MOUPMO  MOU3MO  MOUCH1M  MOUCH3M  CUM3MCH  CUSTMOS  LONGD  CALLW  \
0 0    105      15      99       90      -84        6       11      0      0   
  1    999    1200    1145     -201       55     -146        3      1      1   
  2      5       2       1        3        1        4       23      0      0   
  3    600     456     398      144       58      202       13      0      1   
  4      2       0       0        0        0        0        1      0      0   

     LINES  ...  ARTS_Y  TRAVEL_Domestic  TRAVEL_InterGalactic  \
0 0      2  ...   False            False                 False   
  1      2  ...   False             True                 False   
  2      1  ...    True            False                 False   
  3      1  ...   False            False                 False   
  4      1  ...   False             True                 False   

     TRAVEL_International  EDUC_Bachelors  EDUC_Elementary  EDUC_GED  \
0 0                 False            True         

**ANALYSIS Multilayer perceptron**

In [ ]:
#Construct MultiLayer Perceptron Model
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

mlp = MLPClassifier(max_iter=100)
mlp.fit(X_train1, y_train)
mlp_predict=mlp.predict(X_test1)
print("accuracy Score for MultiLayer Perceptron:{0:6f}".format(mlp.score(X_test1,y_test)))
print("Confusion Matrix for MultiLayer Perceptron:")
print(confusion_matrix(y_test,mlp_predict))

# Calculate Accuracy, Precision, Recall, and F1 score
accuracy = accuracy_score(y_test, mlp_predict)
precision = precision_score(y_test, mlp_predict)
recall = recall_score(y_test, mlp_predict)
f1 = f1_score(y_test, mlp_predict)

# Print results
print("Accuracy Score for MultiLayer Perceptron: {0:6f}".format(accuracy))
print("Precision Score for MultiLayer Perceptron: {0:6f}".format(precision))
print("Recall Score for MultiLayer Perceptron: {0:6f}".format(recall))
print("F1 Score for MultiLayer Perceptron: {0:6f}".format(f1))

accuracy Score for MultiLayer Perceptron:0.781333
Confusion Matrix for MultiLayer Perceptron:
[[283  14]
 [ 68  10]]
Accuracy Score for MultiLayer Perceptron: 0.781333
Precision Score for MultiLayer Perceptron: 0.416667
Recall Score for MultiLayer Perceptron: 0.128205
F1 Score for MultiLayer Perceptron: 0.196078


**ANALYSIS Linear SVC**

In [ ]:
#Construct Linear Support Vector Machine Model
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


linsvm = LinearSVC(max_iter=300)
linsvm.fit(X_train1, y_train)
linsvm_predict=linsvm.predict(X_test1)
print("accuracy Score for Linear SVM Classifier:{0:6f}".format(linsvm.score(X_test1,y_test)))
print("Confusion Matrix for Linear SVM Classifier:")
print(confusion_matrix(y_test,linsvm_predict))

# Calculate Accuracy, Precision, Recall, and F1 Score
accuracy = accuracy_score(y_test, linsvm_predict)

# For binary classification, you don't need the 'average' parameter
precision = precision_score(y_test, linsvm_predict, average='binary')
recall = recall_score(y_test, linsvm_predict, average='binary')
f1 = f1_score(y_test, linsvm_predict, average='binary')


# Print the results
print("Accuracy Score for Linear SVM Classifier: {0:6f}".format(accuracy))
print("Precision Score for Linear SVM Classifier: {0:6f}".format(precision))
print("Recall Score for Linear SVM Classifier: {0:6f}".format(recall))
print("F1 Score for Linear SVM Classifier: {0:6f}".format(f1))

accuracy Score for Linear SVM Classifier:0.912000
Confusion Matrix for Linear SVM Classifier:
[[270  27]
 [  6  72]]
Accuracy Score for Linear SVM Classifier: 0.912000
Precision Score for Linear SVM Classifier: 0.727273
Recall Score for Linear SVM Classifier: 0.923077
F1 Score for Linear SVM Classifier: 0.813559


**ANALYSIS KNN**

In [ ]:
#Construct K-Nearest Neighbor Model
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)

neigh.fit(X_train1, y_train)
neigh_predict=neigh.predict(X_test1)
print("accuracy Score for KNeighborsClassifier:{0:6f}".format(neigh.score(X_test1,y_test)))
print("Confusion Matrix for KNeighborsClassifier:")
print(confusion_matrix(y_test,neigh_predict))

# Calculate Accuracy, Precision, Recall, and F1 Score
accuracy = accuracy_score(y_test, neigh_predict)
precision = precision_score(y_test, neigh_predict, average='binary')  # Use 'binary' for binary classification
recall = recall_score(y_test, neigh_predict, average='binary')  # Use 'binary' for binary classification
f1 = f1_score(y_test, neigh_predict, average='binary')

# Print the results
print("Accuracy Score for KNeighborsClassifier: {0:6f}".format(accuracy))
print("Precision Score for KNeighborsClassifier: {0:6f}".format(precision))
print("Recall Score for KNeighborsClassifier: {0:6f}".format(recall))
print("F1 Score for KNeighborsClassifier: {0:6f}".format(f1))

accuracy Score for KNeighborsClassifier:1.000000
Confusion Matrix for KNeighborsClassifier:
[[297   0]
 [  0  78]]
Accuracy Score for KNeighborsClassifier: 1.000000
Precision Score for KNeighborsClassifier: 1.000000
Recall Score for KNeighborsClassifier: 1.000000
F1 Score for KNeighborsClassifier: 1.000000


**ANALYSIS Decision Tree**

In [ ]:
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY
clf = DecisionTreeClassifier()
clf.fit(X_train1,y_train)
clf_predict=clf.predict(X_test1)
print("accuracy Score for Decision Tree:{0:6f}".format(clf.score(X_test1,y_test)))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(y_test,clf_predict))

# Calculate Accuracy, Precision, Recall, and F1 Score
accuracy = accuracy_score(y_test, clf_predict)
precision = precision_score(y_test, clf_predict, average='binary')  # Use 'binary' for binary classification
recall = recall_score(y_test, clf_predict, average='binary')  # Use 'binary' for binary classification
f1 = f1_score(y_test, clf_predict, average='binary')

# Print the results
print("Accuracy Score for Decision Tree: {0:6f}".format(accuracy))
print("Precision Score for Decision Tree: {0:6f}".format(precision))
print("Recall Score for Decision Tree: {0:6f}".format(recall))
print("F1 Score for Decision Tree: {0:6f}".format(f1))

accuracy Score for Decision Tree:1.000000
Confusion Matrix for Decision Tree
[[297   0]
 [  0  78]]
Accuracy Score for Decision Tree: 1.000000
Precision Score for Decision Tree: 1.000000
Recall Score for Decision Tree: 1.000000
F1 Score for Decision Tree: 1.000000


**Hyperparameter tuning on the Decision Tree classifier using random search**

In [ ]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(X_train1, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(X_train1,y_train)
clf_predict = clf.predict(X_test1)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning for Decision Tree:{0:6f}".format(clf.score(X_test1,y_test)))
print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,clf_predict))
print("=== Classification Report ===")
print(classification_report(y_test,clf_predict))

#get cross-validation report
clf_cv_score = cross_val_score(clf, X_train1, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())

{'min_samples_split': 20, 'max_depth': 7}
accuracy Score (training) after hypertuning for Decision Tree:0.994667
Confusion Matrix after hypertuning for Decision Tree
[[235   0]
 [  2 138]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       235
           1       1.00      0.99      0.99       140

    accuracy                           0.99       375
   macro avg       1.00      0.99      0.99       375
weighted avg       0.99      0.99      0.99       375

=== All AUC Scores ===
[0.9974204  0.99941038 1.         0.99978022 0.99838828 0.99047619
 0.99933036 1.         0.99925595 0.99918155]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.998324332408943


In [ ]:
from scipy.stats import randint

# Define the parameter distribution for random search
param_dist_clf = {
    'max_depth': [5, 10, 20, 30, None],
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20)
}

# Random Search with cross-validation
random_search_clf = RandomizedSearchCV(estimator=clf, param_distributions=param_dist_clf, n_iter=20, cv=5, scoring='accuracy', random_state=42)
random_search_clf.fit(X_train, y_train)

# Best parameters from random search
print("Best parameters from Random Search (Decision Tree):", random_search_clf.best_params_)

Best parameters from Random Search (Decision Tree): {'max_depth': 30, 'min_samples_leaf': 15, 'min_samples_split': 12}


**Feature selection using SelectFromModel**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score

# Step 1: Load the CSV file into a pandas DataFrame
trainData = pd.read_csv(r'/content/sample_data/Cell Phone Churn-TRAIN.csv')
testData = pd.read_csv(r'/content/sample_data/Cell Phone Churn-TEST.csv')

# Step 2: One-Hot Encoding for all columns in train and test data
trainData_encoded = pd.get_dummies(trainData)
testData_encoded = pd.get_dummies(testData)

# Ensure both datasets have the same columns (if there are any mismatches)
trainData_encoded, testData_encoded = trainData_encoded.align(testData_encoded, join='left', axis=1, fill_value=0)

# Step 3: Prepare the dataset
# Assume the last column is the target variable
X = trainData_encoded.iloc[:, :-1]  # Features (all columns except the last one)
y = trainData_encoded.iloc[:, -1]   # Target variable (last column)

# Step 4: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Train a model to get feature importance
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 6: Perform feature selection using SelectFromModel
selector = SelectFromModel(model, threshold="mean", max_features=None)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# Step 7: Evaluate the model performance on selected features
# Train a new model with the selected features
model_selected = RandomForestClassifier(n_estimators=100, random_state=42)
model_selected.fit(X_train_selected, y_train)

# Predict on the test set with the selected features
y_pred = model_selected.predict(X_test_selected)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy with selected features: {accuracy:.4f}")

# Step 8: Get the selected feature names
selected_features = X.columns[selector.get_support()]
print(f"Selected features: {selected_features}")

# Step 9: Save the dataset with the selected features to a new CSV file
X_selected = trainData_encoded[selected_features]
X_selected['target'] = y  # Assuming 'target' is the name of the target column
X_selected.to_csv('selected_features_data.csv', index=False)

print("Dataset with selected features saved to 'selected_features_data.csv'")

Model accuracy with selected features: 1.0000
Selected features: Index(['MOUMO', 'MOU3MO', 'MOUCH1M', 'MOUCH3M', 'CUM3MCH', 'CUSTMOS', 'LINES',
       'INCOME', 'HRS_TV', 'TOTCHNG', 'EDUC_Bachelors', 'EDUC_Masters'],
      dtype='object')
Dataset with selected features saved to 'selected_features_data.csv'


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Step 1: Load the CSV file into a pandas DataFrame
trainData = pd.read_csv(r'/content/sample_data/Cell Phone Churn-TRAIN.csv')
testData = pd.read_csv(r'/content/sample_data/Cell Phone Churn-TEST.csv')

# Step 2: One-Hot Encoding for all columns in train and test data
trainData_encoded = pd.get_dummies(trainData)
testData_encoded = pd.get_dummies(testData)

# Ensure both datasets have the same columns (if there are any mismatches)
trainData_encoded, testData_encoded = trainData_encoded.align(testData_encoded, join='left', axis=1, fill_value=0)

# Step 3: Prepare the dataset
# Assume the last column is the target variable
X = trainData_encoded.iloc[:, :-1]  # Features (all columns except the last one)
y = trainData_encoded.iloc[:, -1]   # Target variable (last column)

# Step 4: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Train the DecisionTreeClassifier model to get feature importance
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# Step 6: Get feature importances from the model
feature_importances = model.feature_importances_

# Step 7: Get the feature names (column names)
feature_names = X.columns

# Step 8: Create a DataFrame to pair feature names with their importance scores
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})

# Step 9: Sort the DataFrame by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Step 10: List the top 5 most important features
top_5_features = feature_importance_df.head(5)
print("Top 5 features based on importance:")
print(top_5_features)

# Step 11: Evaluate the model performance (optional)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {accuracy:.4f}")


Top 5 features based on importance:
   Feature  Importance
14  HRS_TV    0.517928
7    LONGD    0.166066
21   SEX_M    0.163832
15  TOTMOU    0.034278
2   MOU3MO    0.029956
Model accuracy: 1.0000


**Ensemble**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from google.colab import files

# Step 1: Load the CSV files into pandas DataFrames
train_data = pd.read_csv(r'/content/sample_data/Cell Phone Churn-TRAIN.csv')
test_data = pd.read_csv(r'/content/sample_data/Cell Phone Churn-TEST.csv')

# Step 2: One-Hot Encoding for all columns in train and test data
trainData_encoded = pd.get_dummies(train_data)
testData_encoded = pd.get_dummies(test_data)

# Ensure both datasets have the same columns (if there are any mismatches)
trainData_encoded, testData_encoded = trainData_encoded.align(testData_encoded, join='left', axis=1, fill_value=0)

# Step 3: Prepare the datasets
# Assume the last column is the target variable
X_train = trainData_encoded.iloc[:, :-1]  # Features (all columns except the last one)
y_train = trainData_encoded.iloc[:, -1]   # Target variable (last column)

X_test = testData_encoded.iloc[:, :-1]  # Features (all columns except the last one)
y_test = testData_encoded.iloc[:, -1]   # Target variable (last column)

# Step 4: Define base models
decision_tree = DecisionTreeClassifier(random_state=42)
knn = KNeighborsClassifier()
mlp = MLPClassifier(random_state=42, max_iter=1000)
svm = LinearSVC(random_state=42)

# Step 5: Define the meta-model (Logistic Regression)
meta_model = LogisticRegression()

# Step 6: Create the Stacking Classifier
stacked_model = StackingClassifier(
    estimators=[('dt', decision_tree), ('knn', knn), ('mlp', mlp), ('svm', svm)],
    final_estimator=meta_model
)

# Step 7: Train the stacking classifier on the training data
stacked_model.fit(X_train, y_train)

# Step 8: Make predictions on the test data
y_pred = stacked_model.predict(X_test)

# Step 9: Evaluate the performance using Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Step 10: Save the predictions to a CSV file
output = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
output.to_csv('stacked_model_predictions.csv', index=False)

print("Predictions have been saved to 'stacked_model_predictions.csv'")

Test Accuracy: 1.0000
Predictions have been saved to 'stacked_model_predictions.csv'


**Best parameters found through hyperparameter tuning of the Decision Tree classifier**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from google.colab import files

# Step 2: Load the CSV files into pandas DataFrames
train_data = pd.read_csv(r'/content/sample_data/Cell Phone Churn-TRAIN.csv')
test_data = pd.read_csv(r'/content/sample_data/Cell Phone Churn-TEST.csv')


# Step 3: One-hot encoding for categorical columns in train and test data
train_data_encoded = pd.get_dummies(train_data)
test_data_encoded = pd.get_dummies(test_data)

# Ensure both datasets have the same columns (if there are any mismatches)
train_data_encoded, test_data_encoded = train_data_encoded.align(test_data_encoded, join='left', axis=1, fill_value=0)

# Step 4: Prepare the datasets
# Assume the last column is the target variable
X_train = train_data_encoded.iloc[:, :-1]  # Features (all columns except the last one)
y_train = train_data_encoded.iloc[:, -1]   # Target variable (last column)

X_test = test_data_encoded.iloc[:, :-1]  # Features (all columns except the last one)
y_test = test_data_encoded.iloc[:, -1]   # Target variable (last column)

# Step 5: Define the Decision Tree classifier
decision_tree = DecisionTreeClassifier(random_state=42)

# Step 6: Define hyperparameter grid for the Decision Tree classifier
param_grid = {
    'max_depth': [5, 10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

# Step 7: Perform hyperparameter tuning using GridSearchCV
grid_search = GridSearchCV(estimator=decision_tree, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy', verbose=2)

# Step 8: Fit GridSearchCV to the training data
grid_search.fit(X_train, y_train)

# Step 9: Print the best parameters found by GridSearchCV
print("Best Hyperparameters found through GridSearchCV:")
print(grid_search.best_params_)

# Step 10: Construct a Decision Tree classifier with the best parameters
best_tree = grid_search.best_estimator_

# Step 11: Evaluate the model with the test set
y_pred = best_tree.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Step 12: Save the predictions to a CSV file
output = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
output.to_csv('decision_tree_predictions.csv', index=False)

print("Predictions have been saved to 'decision_tree_predictions.csv'")


Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Hyperparameters found through GridSearchCV:
{'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}
Test Accuracy: 1.0000
Predictions have been saved to 'decision_tree_predictions.csv'
